# Regresión de Red elástica

Esta función muestra cómo utilizar TensorFlow para resolver la regresión de red elástica. 
$y = Ax + b$

Usaremos los datos de Iris, específicamente:
```
#  y = Sepal Length
#  x = Pedal Length, Petal Width, Sepal Width
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops

## Configurar para TensorFlow

In [ ]:
ops.reset_default_graph()

# crear el grafo
sess = tf.Session()

## Obtener los datos

In [ ]:
# Cargar los datos
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
iris = datasets.load_iris()
x_vals = np.array([[x[1], x[2], x[3]] for x in iris.data])
y_vals = np.array([y[0] for y in iris.data])

## Configurar el modelo

In [ ]:
# hacer reproducibles los resultados
seed = 13
np.random.seed(seed)
tf.set_random_seed(seed)

# Declarar el tamaño del lote
batch_size = 50

# Inicializar los marcadores de posición
x_data = tf.placeholder(shape=[None, 3], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Crear las variables para la Regresión lineal
A = tf.Variable(tf.random_normal(shape=[3,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

# Declarar las operaciones del modelo
model_output = tf.add(tf.matmul(x_data, A), b)

# Declarar la función de pérdida neta elástica.
elastic_param1 = tf.constant(1.)
elastic_param2 = tf.constant(1.)
l1_a_loss = tf.reduce_mean(tf.abs(A))
l2_a_loss = tf.reduce_mean(tf.square(A))
e1_term = tf.multiply(elastic_param1, l1_a_loss)
e2_term = tf.multiply(elastic_param2, l2_a_loss)
loss = tf.expand_dims(tf.add(tf.add(tf.reduce_mean(tf.square(y_target - model_output)), e1_term), e2_term), 0)

# Declarar el optimizador
my_opt = tf.train.GradientDescentOptimizer(0.001)
train_step = my_opt.minimize(loss)

## Entrenar el modelo

In [ ]:
# Inicializar las variables
init = tf.global_variables_initializer()
sess.run(init)

# Ciclo de entrenamiento
loss_vec = []
for i in range(1000):
    rand_index = np.random.choice(len(x_vals), size=batch_size)
    rand_x = x_vals[rand_index]
    rand_y = np.transpose([y_vals[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss[0])
    if (i+1)%250==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + ' b = ' + str(sess.run(b)))
        print('Loss = ' + str(temp_loss))

## Extraer los resultados del modelo

In [ ]:
# Consigue los coeficientes óptimos
[[sw_coef], [pl_coef], [pw_ceof]] = sess.run(A)
[y_intercept] = sess.run(b)

## Graficar los resultados

In [ ]:
# Graficar la pérdida a través del tiempo
plt.plot(loss_vec, 'k-')
plt.title('Loss per Generation')
plt.xlabel('Generation')
plt.ylabel('Loss')
plt.show()